In [1]:
import gpytorch
import torch

In [19]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


In [20]:
# inicializar el likelihood  y el modelo
likelihood = gpytorch.likelihoods.GaussianLikelihood()
x_train = torch.tensor(pozos_train[["x","y"]].values)
y_train = torch.tensor(pozos_train["norma_embedding"].values)

x_test = torch.tensor(pozos_test[["x","y"]].values)
y_test = torch.tensor(pozos_test["norma_embedding"].values)

model = ExactGPModel(x_train, y_train, likelihood)

In [ ]:
# Encontrar hiperparámetros de modelo óptimos
model.train()
likelihood.train()

# adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1) 

# "Loss" para GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
iters = 25
for i in range(iters):

    optimizer.zero_grad()

    output = model(x_train)

    loss = -mll(output, y_train)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, iters, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()